In [8]:
import pickle

import numpy as np

from Posterior import Posterior
from Util import creation_data

In [9]:
create_data = False
pm_run = True
fb_run = True
em_run = True

n_file = 100
n_data = 100

cfg = {'n_particles': 100, 'theta_eff': None, 'sourcespace': None, 'data': None,
       'n_bins': 50, 'sequence_evolution': None, 'method': None, 'verbose': False}

theta_true = np.linspace(0.01, 0.1, n_file)

err_map_mean_pm, err_map_theta_pm, err_pm_mean_pm, err_pm_theta_pm, cpu_time_pm, ess_pm = np.zeros((6, n_file))
err_map_mean_fb, err_map_theta_fb, err_pm_mean_fb, err_pm_theta_fb, cpu_time_fb, ess_fb = np.zeros((6, n_file))
err_map_mean_em, err_map_theta_em, err_pm_mean_em, err_pm_theta_em, cpu_time_em, ess_em = np.zeros((6, n_file))

DATA CREATION

In [10]:
if create_data:
    for idx, _n in enumerate(theta_true):
        sourcespace, data = creation_data(n_data=n_data, theta=_n)
        with open(f'data/data_{idx}.pkl', 'wb') as f:
            pickle.dump([sourcespace, data, _n], f)

RUN PROP METHOD

In [11]:
for idx, _n in enumerate(theta_true):
    print(f'{idx} - ', end='')
    cfg['theta_eff'] = 0.5 * _n
    with open(f'data/data_{idx}.pkl', 'rb') as f:
        cfg['sourcespace'], cfg['data'], useless = pickle.load(f)
    if pm_run:
        cfg['method'] = 'PM'
        post_pm = Posterior(cfg=cfg)
        post_pm = post_pm.perform_smc()
        with open(f'sol/sol_pm_{idx}.pkl', 'wb') as f:
            pickle.dump(post_pm, f)
    if fb_run:
        cfg['method'] = 'FB'
        post_fb = Posterior(cfg=cfg)
        post_fb = post_fb.perform_smc()
        with open(f'sol/sol_fb_{idx}.pkl', 'wb') as f:
            pickle.dump(post_fb, f)
    if em_run:
        cfg['method'] = 'EM'
        post_em = Posterior(cfg=cfg)
        post_em = post_em.perform_smc()
        with open(f'sol/sol_em_{idx}.pkl', 'wb') as f:
            pickle.dump(post_em, f)

0 - 1 - 

/Users/ale/Documents/phd/toy_example/Particle.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  rapp_like = part_aux.like / self.like


2 - 3 - 

/Users/ale/Documents/phd/toy_example/Particle.py:76: RuntimeWarning: invalid value encountered in double_scalars
  rapp_like = part_aux.like / self.like
/Users/ale/Documents/phd/toy_example/Particle.py:94: RuntimeWarning: invalid value encountered in double_scalars
  rapp_like = part_aux.like / self.like
/Users/ale/Documents/phd/toy_example/Particle.py:94: RuntimeWarning: divide by zero encountered in double_scalars
  rapp_like = part_aux.like / self.like


4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 

KeyboardInterrupt: 

ERRORS EVALUATION

In [ ]:
for idx, _n in enumerate(theta_true):
    print(f'{idx} - ', end='')

    with open(f'sol/sol_pm_{idx}.pkl', 'rb') as f:
        post_pm = pickle.load(f)
    with open(f'sol/sol_fb_{idx}.pkl', 'rb') as f:
        post_fb = pickle.load(f)
    with open(f'sol/sol_em_{idx}.pkl', 'rb') as f:
        post_em = pickle.load(f)

    err_map_mean_pm[idx] = (post_pm.map_mean - 0)
    err_map_mean_fb[idx] = (post_fb.map_mean - 0)
    err_map_mean_em[idx] = (post_em.map_mean - 0)

    err_pm_mean_pm[idx] = (post_pm.pm_mean - 0)
    err_pm_mean_fb[idx] = (post_fb.pm_mean - 0)
    err_pm_mean_em[idx] = (post_em.pm_mean - 0)

    err_map_theta_pm[idx] = (post_pm.map_theta - _n) / _n
    err_map_theta_fb[idx] = (post_fb.map_theta - _n) / _n
    err_map_theta_em[idx] = (post_em.map_theta - _n) / _n

    err_pm_theta_pm[idx] = (post_pm.pm_theta - _n) / _n
    err_pm_theta_fb[idx] = (post_fb.pm_theta - _n) / _n
    err_pm_theta_em[idx] = (post_em.pm_theta - _n) / _n

    cpu_time_pm[idx] = post_pm.cpu_time
    cpu_time_fb[idx] = post_fb.cpu_time
    cpu_time_em[idx] = post_em.cpu_time

    ess_pm[idx] = post_pm.ess
    ess_fb[idx] = post_fb.ess[-1]
    ess_em[idx] = post_em.ess[-1]

with open(f'sol/analytics.pkl', 'wb') as f:
    pickle.dump([err_map_mean_pm, err_map_theta_pm, err_pm_mean_pm, err_pm_theta_pm,
                 err_map_mean_fb, err_map_theta_fb, err_pm_mean_fb, err_pm_theta_fb,
                 err_map_mean_em, err_map_theta_em, err_pm_mean_em, err_pm_theta_em,
                 cpu_time_pm, cpu_time_fb, cpu_time_em, ess_pm, ess_fb, ess_em], f)